In [ ]:
#!pip install selenium
#!pip install webdriver_manager
from selenium import webdriver

from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import codecs
from selenium.webdriver.common.keys import Keys
import re

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

In [ ]:
website = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
path = 'C:/Users/olive/Downloads/chromedriver'
driver = webdriver.Chrome(path)
driver.get(website)

In [ ]:
matches = driver.find_elements(By.CLASS, "lister-item-content")

rang = []
film = []
jahr = []
fsk = []
dauer = []
genre = []
bewertung = []
regisseur = []
stars = []
anzahl_bewertungen = []

for match in matches:
    rang.append(match.find_element(By.XPATH,'//span[@class="lister-item-index unbold text-primary"]').text)
    break
print(rang)

In [ ]:
driver.quit()